# 03 – Hibrit Model Eğitimi

Bu defterde görsel + renk vektörünü birleştiren hibrit modeli eğiteceğiz.

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

from src.models.hybrid_model import HybridColorModel

df_colors = pd.read_csv('data/processed/colors.csv')
df_colors.head()

In [ ]:
# Örnek Dataset sınıfı (gerçek veri setine göre düzenleyin)
class FashionColorDataset(Dataset):
    def __init__(self, df, n_colors=5, transform=None):
        self.df = df.reset_index(drop=True)
        self.n_colors = n_colors
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['image_path']).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)

        cols = [f'c{i}' for i in range(self.n_colors*3)]
        color_vec = row[cols].values.astype('float32')
        # Örnek target (dummy): rastgele skor
        score = np.random.rand()
        y_score = np.float32(score)
        y_class = np.int64(1 if score > 0.5 else 0)

        return img, color_vec, y_score, y_class

In [ ]:
# DataLoader ve model kurulumu (transformlar projeye göre ayarlanmalı)
from torchvision import transforms

train_df, val_df = train_test_split(df_colors, test_size=0.2, random_state=42)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_ds = FashionColorDataset(train_df, transform=transform)
val_ds = FashionColorDataset(val_df, transform=transform)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridColorModel(color_dim=15).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4)
reg_loss_fn = nn.MSELoss()
clf_loss_fn = nn.CrossEntropyLoss()

In [ ]:
# Basit eğitim döngüsü (örnek)
def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0.0
    for imgs, color_vecs, y_scores, y_classes in loader:
        imgs = imgs.to(device)
        color_vecs = color_vecs.to(device)
        y_scores = y_scores.to(device).view(-1, 1)
        y_classes = y_classes.to(device)

        optimizer.zero_grad()
        outputs = model(imgs, color_vecs)

        loss_reg = reg_loss_fn(outputs['score'], y_scores)
        loss_clf = clf_loss_fn(outputs['logits'], y_classes)
        loss = loss_reg + loss_clf

        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)

    return total_loss / len(loader.dataset)

def eval_one_epoch(model, loader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for imgs, color_vecs, y_scores, y_classes in loader:
            imgs = imgs.to(device)
            color_vecs = color_vecs.to(device)
            y_scores = y_scores.to(device).view(-1, 1)
            y_classes = y_classes.to(device)

            outputs = model(imgs, color_vecs)
            loss_reg = reg_loss_fn(outputs['score'], y_scores)
            loss_clf = clf_loss_fn(outputs['logits'], y_classes)
            loss = loss_reg + loss_clf
            total_loss += loss.item() * imgs.size(0)

    return total_loss / len(loader.dataset)

In [ ]:
# Eğitim döngüsünü çalıştırma (dummy örnek)
num_epochs = 3
best_val_loss = float('inf')

for epoch in range(1, num_epochs + 1):
    train_loss = train_one_epoch(model, train_loader, optimizer)
    val_loss = eval_one_epoch(model, val_loader)
    print(f"Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        os.makedirs('results/hybrid', exist_ok=True)
        torch.save(model.state_dict(), 'results/hybrid/best_model.pt')
        print('> Yeni en iyi model kaydedildi.')